# &#x1F50D; Checking segmentation outputs from Organelle-Segmenter-Plugin

### &#x1F4D6; **How to:** 

Advance through each block of code sequentially by pressing `Shift`+`Enter`.

If a block of code contains &#x1F53D; follow the written instructions to fill in the blanks below that line before running it.
```python
#### USER INPUT REQUIRED ###
``` 

________________
## 	**INPUTS**

#### &#x1F3C3; **Run code; no user input required**

&#x1F453; **FYI:** This code block loads all of the necessary python packages and functions you will need for this notebook. Additionally, a [Napari](https://napari.org/stable/) window will open; this is where you will be able to visual the segmentations.

In [1]:
# top level imports
from pathlib import Path
import os, sys
from typing import Optional, Union, Dict, List
import itertools 
import glob

import warnings
import time

import numpy as np
import pandas as pd

import napari

### import local python functions in ../infer_subc
sys.path.append(os.path.abspath((os.path.join(os.getcwd(), '..'))))

from infer_subc.core.file_io import (read_czi_image,
                                        export_inferred_organelle,
                                        import_inferred_organelle,
                                        export_tiff,
                                        list_image_files,
                                        read_tiff_image)



from infer_subc.constants import *
from infer_subc.utils.stats import *
from infer_subc.utils.stats_helpers import *
from infer_subc.utils.stats import _assert_uint16_labels
from infer_subc.core.img import label_uint16


import time
%load_ext autoreload
%autoreload 2

viewer = napari.Viewer()

# TODO: include the file_type option in the main import_inferred_organelle() function
def _import_inferred_organelle(name: str, meta_dict: Dict, out_data_path: Path, file_type: str) -> Union[np.ndarray, None]:
    """
    read inferred organelle from ome.tif file

    Parameters
    ------------
    name: str
        name of organelle.  i.e. nuc, lyso, etc.
    meta_dict:
        dictionary of meta-data (ome) from original file
    out_data_path:
        Path object of directory where tiffs are read from
    file_type: 
        The type of file you want to import as a string (ex - ".tif", ".tiff", ".czi", etc.)

    Returns
    -------------
    exported file name

    """

    # copy the original file name to meta
    img_name = Path(meta_dict["file_name"])  #
    # add params to metadata
    if name is None:
        pass
    else:
        organelle_fname = f"{img_name.stem}-{name}{file_type}"

        organelle_path = out_data_path / organelle_fname

        if Path.exists(organelle_path):
            # organelle_obj, _meta_dict = read_ome_image(organelle_path)
            organelle_obj = read_tiff_image(organelle_path)  # .squeeze()
            print(f"loaded  inferred {len(organelle_obj.shape)}D `{name}`  from {out_data_path} ")
            return organelle_obj
        else:
            print(f"`{name}` object not found: {organelle_path}")
            raise FileNotFoundError(f"`{name}` object not found: {organelle_path}")

__________________________
## **QUALITY CHECK OF SEGMENTATIONS**

#### &#x1F6D1; &#x270D; **User Input Required:**

In [2]:
#### USER INPUT REQUIRED ###
# Copy and paste the paths to the folders where your data is saved inside the quotation marks below. 
# If you have more than one segmentation data folder, include it in the segmentation_data_2 line. If not, type None wihtout quotation marks
# NOTE: for windows, use "/" 
raw_data = "D:/Experiments (C2-117 - current)/C2-121/C2-121_deconvolution"
segmentation_data = "D:/Experiments (C2-117 - current)/C2-121/20230921_C2-121_3D-analysis/20230921_C2-121_segmentation"

location_tosave_edited_segmentations = "D:/Experiments (C2-117 - current)/C2-121/20230921_C2-121_3D-analysis/20240102_C2-121_segmentation-edits"
location_tosave_fullset_gooddata = "D:/Experiments (C2-117 - current)/C2-121/20230921_C2-121_3D-analysis/C2-121_good-segs"

# In quotation marks, include the extension of the file type for your SEGMENTATION and RAW images
raw_file_type = ".tiff"
seg_file_type = ".tiff"

# In quotation marks, write the suffix associated to each segmentation file. If you don't have that image 
mask_suffix = "masks_A"
lyso_suffix = "lyso"
mito_suffix = "mito"
golgi_suffix = "golgi"
perox_suffix = "perox"
ER_suffix = "ER"
LD_suffix = "LD"

In [3]:
#### Optional - USER INPUT REQUIRED ###
# If your segmentations are saved in more than one folder, fill in the information below about the second file location. If not, type None wihtout quotation marks in all of the lines below.
# Copy and paste the paths to the folders where your data is saved inside the quotation marks below. 
segmentation_data_2 = None

# In quotation marks, write the suffix associated to each segmentation file; if 
mask_suffix_2 = None
lyso_suffix_2 = None
mito_suffix_2 = None
golgi_suffix_2 = None
perox_suffix_2 = None
ER_suffix_2 = None
LD_suffix_2 = None

### &#x1F3C3; **Run code; no user input required**

In [4]:
raw_file_list = list_image_files(Path(raw_data),".tiff")

pd.set_option('display.max_colwidth', None)
pd.DataFrame({"Image Name":raw_file_list})

,Image Name
0,D:\Experiments (C2-117 - current)\C2-121\C2-121_deconvolution\20230727_C2-121_conditioned_well 2_cell 1_50uM NaAsO_Linear unmixing_0_cmle.ome.tiff
1,D:\Experiments (C2-117 - current)\C2-121\C2-121_deconvolution\20230727_C2-121_conditioned_well 2_cell 2_50uM NaAsO_Linear unmixing_0_cmle.ome.tiff
2,D:\Experiments (C2-117 - current)\C2-121\C2-121_deconvolution\20230727_C2-121_conditioned_well 2_cell 3_50uM NaAsO_Linear unmixing_0_cmle.ome.tiff
3,D:\Experiments (C2-117 - current)\C2-121\C2-121_deconvolution\20230727_C2-121_conditioned_well 2_cell 4_50uM NaAsO_Linear unmixing_0_cmle.ome.tiff
4,D:\Experiments (C2-117 - current)\C2-121\C2-121_deconvolution\20230727_C2-121_conditioned_well 2_cell 5_50uM NaAsO_Linear unmixing_0_cmle.ome.tiff
5,D:\Experiments (C2-117 - current)\C2-121\C2-121_deconvolution\20230727_C2-121_conditioned_well 3_cell 1_25nM TG_Linear unmixing_0_cmle.ome.tiff
6,D:\Experiments (C2-117 - current)\C2-121\C2-121_deconvolution\20230727_C2-121_conditioned_well 3_cell 2_25nM TG_Linear unmixing_0_cmle.ome.tiff
7,D:\Experiments (C2-117 - current)\C2-121\C2-121_deconvolution\20230727_C2-121_conditioned_well 3_cell 3_25nM TG_Linear unmixing_0_cmle.ome.tiff
8,D:\Experiments (C2-117 - current)\C2-121\C2-121_deconvolution\20230727_C2-121_conditioned_well 3_cell 4_25nM TG_Linear unmixing_0_cmle.ome.tiff
9,D:\Experiments (C2-117 - current)\C2-121\C2-121_deconvolution\20230727_C2-121_conditioned_well 3_cell 5_25nM TG_Linear unmixing_0_cmle.ome.tiff


#### &#x1F6D1; &#x270D; **User Input required:**
&#x1F53C; Use the list  above to determine the index of the image you would like to look at.

In [10]:
#### USER INPUT REQUIRED ###
# Utilizing the list above as reference, change this index number (left column in table) to select a specific image
num = 0

### &#x1F3C3; **Run code; no user input required**

In [11]:
raw_img_data, raw_meta_dict = read_czi_image(raw_file_list[num])
print("Image name:")
print(raw_meta_dict['name'][0].split(" :: ")[0])

mask_seg = _import_inferred_organelle(mask_suffix, raw_meta_dict, Path(segmentation_data), seg_file_type)
lyso_seg = _import_inferred_organelle(lyso_suffix, raw_meta_dict, Path(segmentation_data), seg_file_type)
mito_seg = _import_inferred_organelle(mito_suffix, raw_meta_dict, Path(segmentation_data), seg_file_type)
golgi_seg = _import_inferred_organelle(golgi_suffix, raw_meta_dict, Path(segmentation_data), seg_file_type)
perox_seg = _import_inferred_organelle(perox_suffix, raw_meta_dict, Path(segmentation_data), seg_file_type)
ER_seg = _import_inferred_organelle(ER_suffix, raw_meta_dict, Path(segmentation_data), seg_file_type)
LD_seg = _import_inferred_organelle(LD_suffix, raw_meta_dict, Path(segmentation_data), seg_file_type)

if segmentation_data_2 is not None:
    mask_seg = _import_inferred_organelle(mask_suffix, raw_meta_dict, Path(segmentation_data_2), seg_file_type)
    lyso_seg = _import_inferred_organelle(lyso_suffix, raw_meta_dict, Path(segmentation_data_2), seg_file_type)
    mito_seg = _import_inferred_organelle(mito_suffix, raw_meta_dict, Path(segmentation_data_2), seg_file_type)
    golgi_seg = _import_inferred_organelle(golgi_suffix, raw_meta_dict, Path(segmentation_data_2), seg_file_type)
    perox_seg = _import_inferred_organelle(perox_suffix, raw_meta_dict, Path(segmentation_data_2), seg_file_type)
    ER_seg = _import_inferred_organelle(ER_suffix, raw_meta_dict, Path(segmentation_data_2), seg_file_type)
    LD_seg = _import_inferred_organelle(LD_suffix, raw_meta_dict, Path(segmentation_data_2), seg_file_type)

viewer.layers.clear()
viewer.add_image(raw_img_data[0], name='LD_raw', blending='additive')
viewer.add_image(LD_seg, opacity=0.3, colormap='magenta')
viewer.add_image(raw_img_data[1], name='ER_raw', blending='additive')
viewer.add_image(ER_seg, opacity=0.3, colormap='red')
viewer.add_image(raw_img_data[2], name='GL_raw', blending='additive')
viewer.add_image(golgi_seg, opacity=0.3, colormap='yellow')
viewer.add_image(raw_img_data[3], name='LS_raw', blending='additive')
viewer.add_image(lyso_seg, opacity=0.3, colormap='cyan')
viewer.add_image(raw_img_data[4], name='MT_raw', blending='additive')
viewer.add_image(mito_seg, opacity=0.3, colormap='green')
viewer.add_image(raw_img_data[5], name='PO_raw', blending='additive')
viewer.add_image(perox_seg, opacity=0.3, colormap='bop orange')
viewer.add_image(mask_seg, opacity=0.3)

Image name:
20230727_C2-121_conditioned_well 2_cell 1_50uM NaAsO_Linear unmixing_0_cmle.ome
loaded  inferred 4D `masks_A`  from D:\Experiments (C2-117 - current)\C2-121\20230921_C2-121_3D-analysis\20230921_C2-121_segmentation 
loaded  inferred 3D `lyso`  from D:\Experiments (C2-117 - current)\C2-121\20230921_C2-121_3D-analysis\20230921_C2-121_segmentation 
loaded  inferred 3D `mito`  from D:\Experiments (C2-117 - current)\C2-121\20230921_C2-121_3D-analysis\20230921_C2-121_segmentation 
loaded  inferred 3D `golgi`  from D:\Experiments (C2-117 - current)\C2-121\20230921_C2-121_3D-analysis\20230921_C2-121_segmentation 
loaded  inferred 3D `perox`  from D:\Experiments (C2-117 - current)\C2-121\20230921_C2-121_3D-analysis\20230921_C2-121_segmentation 
loaded  inferred 3D `ER`  from D:\Experiments (C2-117 - current)\C2-121\20230921_C2-121_3D-analysis\20230921_C2-121_segmentation 
loaded  inferred 3D `LD`  from D:\Experiments (C2-117 - current)\C2-121\20230921_C2-121_3D-analysis\20230921_C2-1

<Image layer 'mask_seg' at 0x2831562b580>

### &#x1F6D1; **STOP: Use the `Napari` window to review all of the segmentations for this image.** &#x1F50E;

#### At this point, take note of which segmentations need to be edited, if any. 

#### Once you are finished reviewing the images, continue on to the next sections to 1) Edit the segmentation (if necessary) or 2) Save the final set of segmentations for this image in a new folder. This will make preparing for quantitative analysis much simpler.

__________________________
## **EDITING SEGMENTATIONS**

#### &#x1F6D1; &#x270D; **User Input:**

In [21]:
#### USER INPUT REQUIRED ###
# Indicate which segmentations need editing by typing True. If the segmentations are good and do not need editing, indicate False.
edit_cell = False
edit_nuc = False
edit_LD = False 
edit_ER = False
edit_golgi = False
edit_lyso = False
edit_mito = False
edit_perox = False

### &#x1F3C3; **Run code; no user input required** 
### &#x1F440; **See code block output for instructions**

In [13]:
if edit_cell is True:
    viewer.layers.clear()
    viewer.add_image(raw_img_data)
    viewer.add_labels(mask_seg[1])
    print("Head to the Napari window!")
    print("You can edit your segmentation as needed there.")
    print("Be sure to save the new segmentation using File > Save in the Napari window. You should save it to the folder you listed as 'location_tosave_edited_segmentations'")
elif edit_cell is False:
    print("Continue - run the next block of code")
else:
    print("There is an error somewhere!")

Head to the Napari window!
You can edit your segmentation as needed there.
Be sure to save the new segmentation using File > Save in the Napari window. You should save it to the folder you listed as 'location_tosave_edited_segmentations'


In [14]:
if edit_nuc is True:
    viewer.layers.clear()
    viewer.add_image(raw_img_data)
    viewer.add_labels(mask_seg[2])
    print("Head to the Napari window!")
    print("You can edit your segmentation as needed there.")
    print("Be sure to save the new segmentation using File > Save in the Napari window. You should save it to the folder you listed as 'location_tosave_edited_segmentations'")
elif edit_nuc is False:
    print("Continue - run the next block of code")
else:
    print("There is an error somewhere!")

Continue - run the next block of code


In [15]:
if edit_LD is True:
    viewer.layers.clear()
    viewer.add_image(raw_img_data)
    viewer.add_labels(LD_seg)
    print("Head to the Napari window!")
    print("You can edit your segmentation as needed there.")
    print("Be sure to save the new segmentation using File > Save in the Napari window. You should save it to the folder you listed as 'location_tosave_edited_segmentations'")
elif edit_LD is False:
    print("Continue - run the next block of code")
else:
    print("There is an error somewhere!")

Continue - run the next block of code


In [16]:
if edit_ER is True:
    viewer.layers.clear()
    viewer.add_image(raw_img_data)
    viewer.add_labels(ER_seg)
    print("Head to the Napari window!")
    print("You can edit your segmentation as needed there.")
    print("Be sure to save the new segmentation using File > Save in the Napari window. You should save it to the folder you listed as 'location_tosave_edited_segmentations'")
elif edit_ER is False:
    print("Continue - run the next block of code")
else:
    print("There is an error somewhere!")

Continue - run the next block of code


In [17]:
if edit_golgi is True:
    viewer.layers.clear()
    viewer.add_image(raw_img_data)
    viewer.add_labels(golgi_seg)
    print("Head to the Napari window!")
    print("You can edit your segmentation as needed there.")
    print("Be sure to save the new segmentation using File > Save in the Napari window. You should save it to the folder you listed as 'location_tosave_edited_segmentations'")
elif edit_golgi is False:
    print("Continue - run the next block of code")
else:
    print("There is an error somewhere!")

Continue - run the next block of code


In [18]:
if edit_lyso is True:
    viewer.layers.clear()
    viewer.add_image(raw_img_data)
    viewer.add_labels(lyso_seg)
    print("Head to the Napari window!")
    print("You can edit your segmentation as needed there.")
    print("Be sure to save the new segmentation using File > Save in the Napari window. You should save it to the folder you listed as 'location_tosave_edited_segmentations'")
elif edit_lyso is False:
    print("Continue - run the next block of code")
else:
    print("There is an error somewhere!")

Continue - run the next block of code


In [19]:
if edit_mito is True:
    viewer.layers.clear()
    viewer.add_image(raw_img_data)
    viewer.add_labels(mito_seg)
    print("Head to the Napari window!")
    print("You can edit your segmentation as needed there.")
    print("Be sure to save the new segmentation using File > Save in the Napari window. You should save it to the folder you listed as 'location_tosave_edited_segmentations'")
elif edit_mito is False:
    print("Continue - run the next block of code")
else:
    print("There is an error somewhere!")

Continue - run the next block of code


In [20]:
if edit_perox is True:
    viewer.layers.clear()
    viewer.add_image(raw_img_data)
    viewer.add_labels(perox_seg)
    print("Head to the Napari window!")
    print("You can edit your segmentation as needed there.")
    print("Be sure to save the new segmentation using File > Save in the Napari window. You should save it to the folder you listed as 'location_tosave_edited_segmentations'")
elif edit_perox is False:
    print("Continue - run the next block of code")
else:
    print("There is an error somewhere!")

Continue - run the next block of code


__________________
# **SAVE ALL CORRECT SEGMENTATIONS** - into one folder for quantification

In [23]:
if edit_cell is True:
    cell_seg = _import_inferred_organelle("cell", raw_meta_dict, location_tosave_edited_segmentations, seg_file_type)
    out_file_n = export_inferred_organelle(cell_seg, "cell", raw_meta_dict, Path(location_tosave_fullset_gooddata))
elif edit_cell is False:
    cell_seg = mask_seg[1]
    out_file_n = export_inferred_organelle(cell_seg, "cell", raw_meta_dict, Path(location_tosave_fullset_gooddata))
else:
    print("There is an error somewhere!")

saved file: 20230727_C2-121_conditioned_well 2_cell 1_50uM NaAsO_Linear unmixing_0_cmle.ome-cell-copy


In [ ]:
if edit_nuc is True:
    nuc_seg = _import_inferred_organelle("nuc", raw_meta_dict, location_tosave_edited_segmentations, seg_file_type)
    out_file_n = export_inferred_organelle(cell_seg, "nuc", raw_meta_dict, Path(location_tosave_fullset_gooddata))
elif edit_cell is False:
    nuc_seg = mask_seg[2]
    out_file_n = export_inferred_organelle(cell_seg, "nuc", raw_meta_dict, Path(location_tosave_fullset_gooddata))
else:
    print("There is an error somewhere!")

Head to the Napari window!
You can edit your segmentation as needed there.
Be sure to save the new segmentation using File > Save in the Napari window. You should save it to the folder you listed as 'location_tosave_edited_segmentations'


In [ ]:
if edit_LD is True:
    LD_seg = _import_inferred_organelle("LD", raw_meta_dict, location_tosave_edited_segmentations, seg_file_type)
    out_file_n = export_inferred_organelle(LD_seg, "LD", raw_meta_dict, Path(location_tosave_fullset_gooddata))
elif edit_LD is False:
    out_file_n = export_inferred_organelle(LD_seg, "LD", raw_meta_dict, Path(location_tosave_fullset_gooddata))
else:
    print("There is an error somewhere!")

Head to the Napari window!
You can edit your segmentation as needed there.
Be sure to save the new segmentation using File > Save in the Napari window. You should save it to the folder you listed as 'location_tosave_edited_segmentations'


In [ ]:
if edit_ER is True:
    ER_seg = _import_inferred_organelle("ER", raw_meta_dict, location_tosave_edited_segmentations, seg_file_type)
    out_file_n = export_inferred_organelle(ER_seg, "ER", raw_meta_dict, Path(location_tosave_fullset_gooddata))
elif edit_ER is False:
    out_file_n = export_inferred_organelle(ER_seg, "ER", raw_meta_dict, Path(location_tosave_fullset_gooddata))
else:
    print("There is an error somewhere!")

Head to the Napari window!
You can edit your segmentation as needed there.
Be sure to save the new segmentation using File > Save in the Napari window. You should save it to the folder you listed as 'location_tosave_edited_segmentations'


In [ ]:
if edit_golgi is True:
    golgi_seg = _import_inferred_organelle("golgi", raw_meta_dict, location_tosave_edited_segmentations, seg_file_type)
    out_file_n = export_inferred_organelle(golgi_seg, "golgi", raw_meta_dict, Path(location_tosave_fullset_gooddata))
elif edit_golgi is False:
    out_file_n = export_inferred_organelle(golgi_seg, "golgi", raw_meta_dict, Path(location_tosave_fullset_gooddata))
else:
    print("There is an error somewhere!")

Head to the Napari window!
You can edit your segmentation as needed there.
Be sure to save the new segmentation using File > Save in the Napari window. You should save it to the folder you listed as 'location_tosave_edited_segmentations'


In [ ]:
if edit_lyso is True:
    lyso_seg = _import_inferred_organelle("lyso", raw_meta_dict, location_tosave_edited_segmentations, seg_file_type)
    out_file_n = export_inferred_organelle(lyso_seg, "lyso", raw_meta_dict, Path(location_tosave_fullset_gooddata))
elif edit_lyso is False:
    out_file_n = export_inferred_organelle(lyso_seg, "lyso", raw_meta_dict, Path(location_tosave_fullset_gooddata))
else:
    print("There is an error somewhere!")

Head to the Napari window!
You can edit your segmentation as needed there.
Be sure to save the new segmentation using File > Save in the Napari window. You should save it to the folder you listed as 'location_tosave_edited_segmentations'


In [ ]:
if edit_mito is True:
    mito_seg = _import_inferred_organelle("mito", raw_meta_dict, location_tosave_edited_segmentations, seg_file_type)
    out_file_n = export_inferred_organelle(mito_seg, "mito", raw_meta_dict, Path(location_tosave_fullset_gooddata))
elif edit_mito is False:
    out_file_n = export_inferred_organelle(mito_seg, "mito", raw_meta_dict, Path(location_tosave_fullset_gooddata))
else:
    print("There is an error somewhere!")

Head to the Napari window!
You can edit your segmentation as needed there.
Be sure to save the new segmentation using File > Save in the Napari window. You should save it to the folder you listed as 'location_tosave_edited_segmentations'


In [ ]:
if edit_perox is True:
    perox_seg = _import_inferred_organelle("perox", raw_meta_dict, location_tosave_edited_segmentations, seg_file_type)
    out_file_n = export_inferred_organelle(perox_seg, "perox", raw_meta_dict, Path(location_tosave_fullset_gooddata))
elif edit_perox is False:
    out_file_n = export_inferred_organelle(perox_seg, "perox", raw_meta_dict, Path(location_tosave_fullset_gooddata))
else:
    print("There is an error somewhere!")

Head to the Napari window!
You can edit your segmentation as needed there.
Be sure to save the new segmentation using File > Save in the Napari window. You should save it to the folder you listed as 'location_tosave_edited_segmentations'


__________________________
## **QUANTIFICATION**

#### **Batch process quantification:**
After all segmentations for all organelles and masks are correctly assembled into one folder per experiment, you can begin quantification. The output will include ***`per object`*** measurements. Use the function below to input the correct folder paths to your data and settings you'd like to use for analysis.

#### &#x1F6D1; &#x270D; **User Input:**

In [ ]:
seg=batch_process_quantification(out_file_name= "20231117_prelim",
                                  seg_path="C:/Users/zscoman/Documents/Python Scripts/Infer-subc-2D/out",
                                  out_path="C:/Users/zscoman/Documents/Python Scripts/Infer-subc-2D/data/test", 
                                  raw_path="C:/Users/zscoman/Documents/Python Scripts/Infer-subc-2D/raw/shannon",
                                  raw_file_type = ".tiff",
                                  organelle_names = ['LD', 'ER', 'golgi', 'lyso', 'mito', 'perox'],
                                  organelle_channels= [0,1,2,3,4,5],
                                  region_names= ['nuc', 'cell'],
                                  masks_file_name= ['nuc', 'cell'],
                                  mask= 'cell',
                                  dist_centering_obj='nuc', 
                                  dist_num_bins=5,
                                  dist_center_on=False,
                                  dist_keep_center_as_bin=True,
                                  dist_zernike_degrees=None,
                                  include_contact_dist= True,
                                  scale=True,
                                  seg_suffix="-",
                                  splitter='X')

#### **Summarize data per cell:**
The function below takes in the quantification results output from `batch_process_quantification()` from one or more experimental replicates. The results will be aggregated and summarized per cell.

In [ ]:
out=_batch_summary_stats(csv_path_list=["C:/Users/zscoman/Documents/Python Scripts/Infer-subc-2D/data/test"],
                         out_path="C:/Users/zscoman/Documents/Python Scripts/Infer-subc-2D/data/test/sumstat",
                         out_preffix="20231117_prelim_",
                         splitter="X")